In [1]:
%load_ext autoreload
%autoreload

import os
import sys
import random
import django
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from datetime import datetime
import time
from tqdm import tqdm_notebook

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.apps import apps
from django_currentuser.middleware import get_current_user

In [ ]:
guestboo = m.Boo.guestboo

for post in tqdm_notebook(m.Post.objects.order_by('-created_at')[20:]):
    
    for i in range(random.randrange(10,20)):
#         post.vote(0, guestboo, note=time.time())
        m.Flager.vote_up(post, guestboo, time.time())
        
    for i in range(random.randrange(10,20)):
#         post.vote(1, guestboo, note=time.time())
        m.Flager.vote_down(post, guestboo, time.time())
#     break

    post.nvotes_up = post._nvotes_up
    post.nvotes_down = post._nvotes_down
    post.save()
    
#     break

<ipython-input-13-90bf6c7e2089>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for post in tqdm_notebook(m.Post.objects.order_by('-created_at')[20:]):


  0%|          | 0/485 [00:00<?, ?it/s]

In [12]:
for post in tqdm_notebook(m.PostQA.objects.order_by('-created_at')):
    flags = m.Flager.objects.filter(object_id=post.id, status=1)
    flags.delete()
#     break

<ipython-input-12-301bcbfb1a04>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for post in tqdm_notebook(m.PostQA.objects.order_by('-created_at')):


  0%|          | 0/200 [00:00<?, ?it/s]

# MaskBase

In [2]:
#m.MaskBase.objects.all().delete()
m.MaskBase.objects.all()

<QuerySet []>

In [3]:
dir_masks = Path(parentdir) / 'getch/static/materials/masks'

In [4]:
for d in dir_masks.glob('*'):
    if d.name != 'archives':
        _type = d.name.upper()
        
        for f in d.rglob('*.*'):
            cat = f.parts[-2]
            
            if cat != 'raw':
                maskbase = m.MaskBase.objects.create(type=_type, category=cat)
                maskbase.pix.save(f.name, ContentFile(f.read_bytes()), save=True)

# MaskSerializer

In [7]:
class EyeMaskSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.EyeMask
        fields = ['id', 'maskbase', 'top', 'left', 'width', 'height']
        read_only_fields = ['id']
        
class MouthMaskSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.MouthMask
        fields = ['id', 'maskbase', 'top', 'left', 'width', 'height']
        read_only_fields = ['id']        
        
EyeMaskSerializer()

EyeMaskSerializer():
    id = IntegerField(read_only=True)
    maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all(), required=False)
    top = FloatField(required=False)
    left = FloatField(required=False)
    width = FloatField(required=False)
    height = FloatField(required=False)

## Serializing

In [8]:
eyemask = m.EyeMask.objects.first()
data = EyeMaskSerializer(eyemask).data; data

{'id': 1, 'maskbase': 1, 'top': 0.0, 'left': 0.0, 'width': 100.0, 'height': 20.0}

## Deserializing
### 1. Create
* serializer로 creation 하지 않는다
* profile 생성할때 mask(1-1관계)는 default로 무조건 같이 생성되며, 이때 Mask.objects.create()를 쓴다

In [2]:
eyemask = m.EyeMask.objects.create(); eyemask;

In [9]:
# serializer = MaskSerializer(data={'maskbase':15, 'top':10, 'left':0, 'width':50, 'height':50})
# serializer.is_valid()
# mask = serializer.save(); mask

### 2. Update

In [12]:
serializer = EyeMaskSerializer(eyemask, data={'maskbase':15, 'top':10, 'left':0, 'width':50, 'height':50})
serializer.is_valid()
eyemask = serializer.save(); eyemask

<EyeMask: EYE | cartoon_m | 2020-08-11_095158.942119SIDEB_EYEMASK_M_CARTOON_05_onepunman.jpg | T=10.0, L=0.0, W=50.0, H=50.0>

# ProfileSerializer

In [5]:
class ProfileSerializer(serializers.ModelSerializer):
    #eyemask = EyeMaskSerializer(required=False)
    #mouthmask = MouthMaskSerializer(required=False)

    class Meta:
        model = m.Profile
        fields = ['id', 'type', 'pix', 'character', 'image', 'text']#, 'eyemask', 'mouthmask']
        read_only_fields = ['id']
        
ProfileSerializer()

ProfileSerializer():
    id = IntegerField(read_only=True)
    type = ChoiceField(choices=(('CHARACTER', 'character'), ('IMAGE', 'image'), ('TEXT', 'text')), required=False)
    pix = ImageField(allow_null=True, max_length=500, required=False)
    character = PrimaryKeyRelatedField(allow_null=True, queryset=Character.objects.all(), required=False)
    image = ImageField(allow_null=True, max_length=500, required=False)
    text = CharField(allow_blank=True, allow_null=True, max_length=10, required=False)

## Serializing

In [6]:
profile = m.Profile.objects.last()
data = ProfileSerializer(profile).data; data

{'id': 34, 'type': 'TEXT', 'pix': 'https://storage.googleapis.com/sideb-proejct.appspot.com/user/sejin76%40nate.com/2020-08-13_195919.813847__sejin76nate.com_mypix.png?Expires=1597995451&GoogleAccessId=sideb-serviceaccount%40sideb-proejct.iam.gserviceaccount.com&Signature=fsVsSTtjcW2gkYXXYwX6wx%2BmOQgmp%2BSZTCYkoe9W8KJGiYFHw7zK19Ljycw3JGjwpRMdlHqfenfG16mTpDrnolfeuGz5tWRec%2FOHVOHZFxOEcIokGVhspuwIcr8dCQ4Ng9B37TqQCv%2F3tnNyeoEFEtsJNPwzGcoiY1SWLX9z1iiCJHXDDf%2Bgty4JlP%2FeF4f%2Fw2Eihz23jFxbOwwi3xuzpk0MOauERQg%2FueACvqdB5s46LrSHgeKwLBFY2jiNnLTSSW%2B09VgEUxOQHF4Kcqrz%2F0tRN9nmzakgmdCzKxu4xMUpd9RL2L9GrblcD9VkIVMq99287Jnu56lxz34WfkPXEQ%3D%3D', 'character': 1, 'image': None, 'text': '강'}

## Deserializing
### 1. create

In [15]:
pix = m.PostVoteOX.objects.all()[0].pix; pix

<ImageFieldFile: post/2020/7/22/sejin76@nate.com/2020-07-22_195937.085170__AB_anorak_B.jpg>

In [17]:
serializer = ProfileSerializer(data={'type':'IMAGE', 'text':'TESTTEST', 'pix':None, 'image':None, 'eye_mask':None, 'mouth_mask':None})
serializer.is_valid()
# profile = serializer.save(); profile

False

In [19]:
serializer.validated_data

{}

In [2]:
profile = m.Profile.objects.create()

In [4]:
profile.pix

<ImageFieldFile: material/profile.png>

In [2]:
user = m.User.objects.get(pk=1)
m.Boo.objects.create(user=user)

<Boo: sideb | sideb@sideb.com>

# PostSerializer

In [7]:
class AuthorSerializer(serializers.ModelSerializer):
    pix = serializers.SerializerMethodField()
    
    class Meta:
        model = m.Boo
        fields = ['id', 'nick', 'text', 'pix']
        read_only_fields = fields

    def get_pix(self, obj):
        return obj.profile.pix.url
    

class PostVoteOXSerializer(serializers.ModelSerializer):
    boo = AuthorSerializer()
    
    class Meta:
        model = m.PostVoteOX
        fields = ['id', 'boo', 'text', 'pix']
        read_only_fields = ['id']
        
class PostVoteABSerializer(serializers.ModelSerializer):
    boo = AuthorSerializer()
    
    class Meta:
        model = m.PostVoteAB
        fields = ['id', 'boo', 'text', 'pix_a', 'pix_b']
        read_only_fields = ['id']
    
class PostSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.Post
        fields = '__all__'
        
    def to_representation(self, instance):
        if isinstance(instance, m.PostVoteOX):
            return PostVoteOXSerializer(instance=instance).data
        
        elif isinstance(instance, m.PostVoteAB):
            return PostVoteABSerializer(instance=instance).data
        
    def create(self, validated_data):
        _type = self.initial_data.pop('type')
        
        if _type == 'postvoteox':
            ser = PostVoteOXSerializer(data=self.initial_data, partial=True)
        
        elif _type == 'postvoteab':
            ser = PostVoteABSerializer(data=self.initial_data, partial=True)

        if ser.is_valid():
            return ser.save()
        
        else:
            print('something wrong when creating post data')
            return None
    
    
    def update(self, instance, validated_data):
        _type = instance.__class__.__name__.lower()
        
        if _type == 'postvoteox':
            ser = PostVoteOXSerializer(instance, data=self.initial_data, partial=True)
        
        elif _type == 'postvoteab':
            ser = PostVoteABSerializer(instance, data=self.initial_data, partial=True)
        
        if ser.is_valid():
            ser.save()
        
        else:
            print('something wrong when updating post data')
        
        return instance
    
            
PostVoteOXSerializer()
PostSerializer()

PostSerializer():
    id = IntegerField(read_only=True)
    text = CharField(allow_blank=True, allow_null=True, max_length=500, required=False, style={'base_template': 'textarea.html'})
    created_at = DateTimeField(read_only=True)
    boo = PrimaryKeyRelatedField(allow_null=True, queryset=Boo.objects.all(), required=False)

## Serializing

In [8]:
posts = m.Post.objects.select_subclasses()
data = PostSerializer(posts, many=True).data; data[0]

{'id': 8, 'boo': OrderedDict([('id', 6), ('nick', '사이드비'), ('text', ''), ('pix', 'https://storage.googleapis.com/sideb-proejct.appspot.com/user/sideb%40sideb.com/2020-08-12_183659.151781__einstein.jpg?Expires=1597995517&GoogleAccessId=sideb-serviceaccount%40sideb-proejct.iam.gserviceaccount.com&Signature=QJgLrW78witNmxVqn7IryVpbRmMMHWVDyHRPTEelAdrMTe7hWmr79aZU0A7xAxRnAZIEU%2BlvyfvfSMXm%2BVahU5rnwADClUWQLp98qNlHE0M8S8HK0FDbkShFJBtxn97rooUAJk9Get6lEi3aWZ5dM92goCoXiPu%2FNkWztvzggwSVMob5j1Lo3HZbHktDUs3cyFk0O3g4mzqmvtAYCcUFale1U%2FrqGUC6%2BP0qm4MdYNEy3A5MADB%2FLZsGkUFgPnk6gv5MBmb8Vqgg8r28%2BEyTuhfRdNyRZgRlLIszi%2FVd%2F2re%2FrkrDonWdWRhyYYHReZoNoLLS%2B%2FpSQUa%2FI7R6kCRHQ%3D%3D')]), 'text': '나에게로의 여행\r\n어때, 즐길 준비 됐어?\r\n#HEY SIDE-b', 'pix': 'https://storage.googleapis.com/sideb-proejct.appspot.com/post/2020/8/20/sideb%40sideb.com/2020-08-20_143347.549448__sideb_logo.jpg?Expires=1597995517&GoogleAccessId=sideb-serviceaccount%40sideb-proejct.iam.gserviceaccount.com&Signature=bdn6mRvKUI%2FuXqaS

In [52]:
posts[1]

<PostVoteOX: OX | >

## Deserializing
### 1. Create

In [53]:
ser = PostSerializer(data={'type':'postvoteab', 'text':'얼레리꼴레리', 'pix_a':posts[0].pix, 'pix_b':posts[1].pix}, partial=True)
ser.is_valid()
post = ser.save(); post

<PostVoteAB: AB | >

### 2. Update

In [47]:
post

<PostVoteOX: OX | >

In [50]:
ser = PostSerializer(post, data={'boo':15, 'text':'자나깨나뽕#으엉으엉우엉'}, partial=True)
ser.is_valid()
post = ser.save(); post

<PostVoteOX: OX | >

In [2]:
m.Post.objects.select_subclasses()

<InheritanceQuerySet [<PostVoteOX: OX | 2020-08-19 08:57:48.072436+00:00>, <PostVoteOX: OX | 2020-08-20 02:14:46.581017+00:00 | 모지리왕# | sejin76@nate.com>, <PostVoteOX: OX | 2020-08-20 02:16:36.069362+00:00 | 모지리왕# | sejin76@nate.com>, <PostVoteAB: AB | 2020-08-20 04:51:03.216220+00:00>]>

In [7]:
_post = m.Post.objects.select_subclasses()[0]; _post

<PostVoteOX: OX | 2020-08-20 05:33:47.195414+00:00 | 모이버 | moiber@moiber.com>

In [9]:
m.Postpix.objects.create(post=_post, img=_post.pix)

<Postpix: Postpix object (2)>